## pricing data paper
In their paper, they assigned one price for the whole database, and didn't use their **weithed coverage** function. 

>The default way to
assign these weights when the data seller has provided at
the minimum the price P of the full database is to assign
equal weight $w_i = \frac{P}{|S|}$ to each instance in the support set.

>Their framework allows the seller to specify additional price points as a set of pairs (Q,p), which is on query level. It means the pricing point is binded with the query. 

>QIRANA can incorporate these price points into the price by assigning different weightes to the instances of the support set.

In [39]:

class UpdateValue():
    '''
    Obj UpdateValue represents the basic tupple in the dataframe.
    '''
    def __init__(self, id, row,column,oldValue, newValue):
        self.id = id
        self.row=row
        self.column=column
        self.oldValue=oldValue
        self.newValue = newValue
        
    def setRow(self,row):
        self.row=row

    def getRow(self):
        return self.row

    def setColumn(self, column):
        self.column = column

    def getColumn(self):
        return self.column

    def setNewValue(self, value):
        self.newValue = value

    def getNewValue(self):
        return self.newValue

    def getId(self):
        return self.id

    def setId(self, id):
        self.id = id
        
    def setOldValue(self, value):
        self.oldValue = value
        
    def getOldValue(self):
        return self.oldValue
    
    def __str__(self):
        r = ['id = ',str(self.id), ', row = ',str(self.row), ', column = ', str(self.column), ', oldValue = ', str(self.oldValue), ', newValue = ', str(self.newValue)]
        return ''.join(r)
        
u = UpdateValue(100, 1 , 1, 'old', 'new')
print(u.getColumn())
str(u)
u.getId()

1


100

1. 先生成updatelist, 不管是否是 consistent的
2. 对于每个update 进行consistency check, 如果不符合, 就去掉, 然后重新生成

In [75]:
import pandas as pd
from IPython.display import display, HTML


path='./data/test1/5gdb.csv'
df=pd.read_csv(path,header=0)
display(df)


columnUniqueValue = {}
for column in df:
    print (df[column].unique())
    columnUniqueValue[column] = df[column].unique()
columnUniqueValue
# display(df.ix[1:2])
# display(df.ix[[1],[2]])
# display(df.columns[1])
# display(df.ix[[0],[0]])
# print (df.iat[0,0])
# print (df.at[1,'PID'])

,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,female,44,fatigue,Advil,cold
1,789,female,68,fatigue,Clofarabine,leukemia
2,739,male,52,fever,Advil,incluenza
3,512,male,73,fatigue,Clofarabine,leukemia


[928 789 739 512]
['female' 'male']
[44 68 52 73]
['fatigue' 'fever']
['Advil' 'Clofarabine']
['cold' 'leukemia' 'incluenza']


{'AGE': array([44, 68, 52, 73], dtype=int64),
 'DRUG': array(['Advil', 'Clofarabine'], dtype=object),
 'GEN': array(['female', 'male'], dtype=object),
 'ILLNESS': array(['cold', 'leukemia', 'incluenza'], dtype=object),
 'PID': array([928, 789, 739, 512], dtype=int64),
 'SYMP': array(['fatigue', 'fever'], dtype=object)}

In [53]:
## generate support set 

def createUpdateCell(df, updateNum, attrNum = 1):
    """
    given a domain dictionary of df, generate the update list.
    
    Args:
        df: the original database instance
        updateNum: the number of updates, the size of update list
        attrNum: the numer of attribute that will be updated, how many attributes will be changed in one update
    Return:
        a list of update obj
    """
    
#     import numpy as np
    import random
    
    # generate the column domain value
    columnDomain = {}
    for column in df:
        columnDomain[column] = df[column].unique()
    
    #  number of rows
    rowSize = df.shape[0]
    colSize = df.shape[1]
    
    updateList = []
    
    for i in range(updateNum
                  
                  ):
        row = i%rowSize
        column = random.randint(0,colSize-1)
        oldValue = str(df.iat[row,column]).strip()
        newValue = random.choice(columnDomain[df.columns[column]])

        u = UpdateValue(i, row, column, oldValue, newValue)
        updateList.append(u)
    return updateList


# test update 当小于总size 的时候
updateList = createUpdateCell(df, 3)
for e in updateList:
    print (e)

print ('*'*30)
# 大于 df 的size的时候
updateList2 = createUpdateCell(df,20)
for e in updateList2:
    print(e)

id = 0, row = 0, column = 3, oldValue = fatigue, newValue = fever
id = 1, row = 1, column = 1, oldValue = female, newValue = male
id = 2, row = 2, column = 3, oldValue = fever, newValue = fatigue
******************************
id = 0, row = 0, column = 1, oldValue = female, newValue = female
id = 1, row = 1, column = 2, oldValue = 68, newValue = 44
id = 2, row = 2, column = 5, oldValue = incluenza, newValue = incluenza
id = 3, row = 3, column = 0, oldValue = 512, newValue = 512
id = 4, row = 0, column = 3, oldValue = fatigue, newValue = fatigue
id = 5, row = 1, column = 2, oldValue = 68, newValue = 44
id = 6, row = 2, column = 1, oldValue = male, newValue = female
id = 7, row = 3, column = 0, oldValue = 512, newValue = 512
id = 8, row = 0, column = 1, oldValue = female, newValue = female
id = 9, row = 1, column = 5, oldValue = leukemia, newValue = incluenza
id = 10, row = 2, column = 1, oldValue = male, newValue = female
id = 11, row = 3, column = 3, oldValue = fatigue, newValue = fati

In [54]:
## apply update to df

def applyUpdateCell(df, updateCell):
    """
    apply the update cell to database instance
    Args:
        df: original database instance
        updateCell: the cell that will be applied
    """
#     newInstance = df.copy(deep = True)
    df.ix[[updateCell.getRow()],[updateCell.getColumn()]] = updateCell.getNewValue()
    return df

def rollBack(df, updateCell):
    df.ix[[updateCell.getRow()],[updateCell.getColumn()]] = updateCell.getOldValue()
    return df

# test applyUpdateCell
# display(df)
u3 = createUpdateCell(df,1)
print (str(u3[0]))
print ('below is original instance')
display(df)
df_new = applyUpdateCell(df,u3[0])
print ('below is new instance')
display(df_new)
df_old = rollBack(df_new,u3[0])
print ('roll back to  old instance')
display(df_old)

id = 0, row = 0, column = 4, oldValue = Advil, newValue = Clofarabine
below is original instance


,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,female,44,fatigue,Advil,cold
1,789,female,68,fatigue,Clofarabine,leukemia
2,739,male,52,fever,Advil,incluenza
3,512,male,73,fatigue,Clofarabine,leukemia


below is new instance


,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,female,44,fatigue,Clofarabine,cold
1,789,female,68,fatigue,Clofarabine,leukemia
2,739,male,52,fever,Advil,incluenza
3,512,male,73,fatigue,Clofarabine,leukemia


roll back to  old instance


,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,female,44,fatigue,Advil,cold
1,789,female,68,fatigue,Clofarabine,leukemia
2,739,male,52,fever,Advil,incluenza
3,512,male,73,fatigue,Clofarabine,leukemia


In [91]:
## query on the database
def getQueryResult(qry, df_local):
    '''
    这里不用传入 df, 是因为在qry中会指定df, 比如 from df 和 from df_new 是不一样的
    sqldf 会 lookup 全局 variable, 然后根据名字来确定是在哪个dataframe 中找
    Args:
        qry: a string with sqlite query format
    Return:
        result in the form of df
    '''
    df = df_local
    from pandasql import sqldf
#     pysqldf = lambda q: sqldf(q, locals())
#     result =pysqldf(qry)
#     print (locals())
    result = sqldf(qry, locals())
    return result
qry1="select SYMP from df where GEN='female' ;"
qry2 = "select ILLNESS from df;"
# qry3 = "select SYMP from df where DRUG = 'Clofarabine' ;"
display(df_old)
result1 = getQueryResult(qry1, df_old)
result2 = getQueryResult(qry2, df_old)
df_new = applyUpdateCell(df_old, u3[0])
result3 = getQueryResult(qry2,df_new)


# print (type(result1))
display(result1)
print ('below is old result')
display(result2)
print ('below is new result')
display(result3)


## 以下测试 change the value in result, won't change the value in original df
result1.iat[1,0] = 1
display(result1)
display(df_old)

,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,female,44,fatigue,Clofarabine,cold
1,789,female,68,fatigue,Clofarabine,leukemia
2,739,male,52,fever,Advil,incluenza
3,512,male,73,fatigue,Clofarabine,leukemia


,SYMP
0,fatigue
1,fatigue


below is old result


,ILLNESS
0,cold
1,leukemia
2,incluenza
3,leukemia


below is new result


,ILLNESS
0,cold
1,leukemia
2,incluenza
3,leukemia


,SYMP
0,fatigue
1,1


,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,female,44,fatigue,Clofarabine,cold
1,789,female,68,fatigue,Clofarabine,leukemia
2,739,male,52,fever,Advil,incluenza
3,512,male,73,fatigue,Clofarabine,leukemia


## locals() vs. globals()

locals() and globals() are python built-in functions that are used to return the corresponding namespace.

In Python , Namespace is a way to implement scope. So global namespace means global scope, so variables(names) defined there are visible throughout the module.

local namepsace is the namespace that is local to a particular function.

globals() returns a dictionary representing the current global namespace.

locals()'s return depends on where it is called, when called directly inside the script scope (not inside a particular function) it returns the same dictionary as globals() that is the global namespace. When called inside a function it returns the local namespace.

In pandasql , the second argument you need to pass is basically this namespace (dictionary) that contains the variables that you are using in the query. That is lets assume you create a DataFrame called a , and then write your query on it. Then pandasql needs to know the DataFrame that corresponds to the name a for this it needs the local/global namespace, and that is what the second argument is for.

So you need to decide what to pass in, example , if your DataFrame is only defined inside a function and does not exist in global scope, you need to pass in locals() return dictionary, If your DataFrame exists in global scope, you need to pass in result of globals() .


这里的locals(), 是指在这个函数内部定义的变量的名字, 比如我的query中都写 `from df` 但是, 如果我定义为locals(), 那么我可以在传入的 query中增加一个参数, 也就是 df_old 或者 df_new, 然后把这个参数传入到 query()方法内部, 并assign 给 df, 这样就可以实现 指定 query, 指定 df 的查询了

In [56]:
result4 = getQueryResult(qry1, df_old)
print (result1.equals(result4))
print (result2.equals(result3))

True
True


In [86]:
## generate price without optimization (directly check whether the result of queries)
def checkUpdateDirect(updateList, q, df):
    """
    Main idea: directly apply updates to instance, check whether the results are equal
    Args:
        updateList: a list of updates
        q: input query
        df: original database instance
    Return:
        a list of updates, whose query results are not equal to original instance 
        (这些 update 生成的instance 和 original instance df 在query 下的 answer 不相等)
    """
    df_old = df
    originalResult = getQueryResult(q,df_old)
    unequalList = []
    for update in updateList:
        df_new = applyUpdateCell(df_old, update)
        if not originalResult.equals(getQueryResult(q, df_new)):
            unequalList.append(update)
        df_old = rollBack(df_new, update)
    return unequalList

# test checkUpdateDirect
print ('below is df_old')
display(df_old)
print ('below is df_new')
display(df_new)
print ('below is for qry1')
for e in checkUpdateDirect(u3, qry1, df):print(e)

print ('below is qry2') 
for e in checkUpdateDirect(u3, qry2, df):print(e)


below is df_old


,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,female,44,fatigue,Advil,cold
1,789,female,68,fatigue,Clofarabine,leukemia
2,739,male,52,fever,Advil,incluenza
3,512,male,73,fatigue,Clofarabine,leukemia


below is df_new


,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,female,44,fatigue,Advil,cold
1,789,female,68,fatigue,Clofarabine,leukemia
2,739,male,52,fever,Advil,incluenza
3,512,male,73,fatigue,Clofarabine,leukemia


below is for qry1


NameError: name 'getQueryResult' is not defined

In [72]:
# from cvxpy import *
import cvxpy as cv
# Create two scalar optimization variables.

X = cv.Variable(3)

indexList = [0,1,2]

# method1:
P = 0
for i in indexList:
    P = X[i] + P

# method2:    
Q = sum([X[i] for i in indexList])    

constraints = [
    sum([X[i] for i in indexList]) == 9,
#     P >= 9,
# X[0] + X[1] + X[2] >= 9,
# X[0] + X[1] == 8,
# X[1] + X[2] <= 8
# X[0] + 3*X[1] == 8,
# 0 <= X, X <= 1
]

# Form objective.
# cost = sum_entries(X)
cost = sum(cv.entr(X))
obj = cv.Maximize(cost)
# Form and solve problem.
prob = cv.Problem(obj, constraints)
prob.solve() # Returns the optimal value.
print ("status:", prob.status)
print ("optimal value", prob.value)
print ("optimal var", X.value)
print (type(X), type(X.value))

status: optimal
optimal value -9.887510571693992
optimal var [[ 3.]
 [ 3.]
 [ 3.]]
<class 'cvxpy.expressions.variables.variable.Variable'> <class 'numpy.matrixlib.defmatrix.matrix'>


## how to price the query

给定的输入是 support set $S$, 一些列 query Q 和对应的 price point p. 输出是 weight w. 

1. 假设support set 是 $S = \{D_1,D_2, ...D_{10}\}$, 给定的一个query $Q$, 对应的 price point $p$
2. $Q(S) \ne Q(D)$ 对于这个set中, 在当前Q下, 找出和 original D 不一样的所有的instance, 这些 instance 组成了price point. 比如 $D_1,D_3,D_5$在$Q$下的结果和 $D$ 不一样, 那么 $p = w_1 + w_3 + w_5$ 这是一个 constraint
3. 为了 optimization, 不需要真的 apply $Q$ 到 $S$的每个instance上, 只用 check update $U_i$ 是否 change 了 $Q(D)$ 的结果

In [69]:
# TODO: give the query() method, exp_level default parameter, where the exp_level = ground_level
# 给 update 进行编号 内部 unique id, 从 0 开始

def checkUpdate(updateList, q, df):
    """
    Main idea: 为了优化, 不用把update 真的apply 到instance上, 而只用check update 是否更改了query answer 返回的那块数据
    Args:
        updateList, 包含了所有的updates
        q: 输入的一个query
        df: original database instance
    Return:
        a list of updates, 这些 update 生成的instance 和 original instance df 在query 下的 answer 不相等.
    """
    originalResult = query(q)
    unequalList = []
    for update in updateList:
        if checkChange(update, originalResult, df):
            unequalList.append(update)
    return unequalList

def checkChange(update, originalResult, df):
    """
    Main idea:
        check whether the update is in the original result area
    Args:
        update: one update
        originalResult: the answer of query on df, it should include the position (row, column) and the value information
        df: original database instance
    Return:
        True, if the update belongs to original result; otherwise, False
    """
    pass

def constructConstraints(updateList, df, queryList, pointList):
    """
    Main idea:
        construct the constraints of the convex optimization
    
    Return:
        a list of constraint, each of them is a tuple pair, which contain (unquealList, point)
    """
    # query, point pairs
    pairwise = zip(queryList, pointList)
    
    # construct constraints
    constraintList = []
    for element in pairwise:
        unequalList = checkUpdateDirect(updateList, element[0], df)
        ids = [i.getId() for i in unequalList]
        constraintList.append((ids, element[1]))
    return constraintList
        
def calculateWeight(updateList, df, queryList, pointList):
    """
    Main idea:
        given a list of database instance (updateList), a bundle of queries and corresponding price points, calculate the weight
    for each instance in the support set (updateList)

    Args:
        updateList
        queryList, 用户指定
        pointList, 用户指定
        df, original database instance
    Return:
        a list of weights for each instance in the updateList
    """
    
    import cvxpy as cv
    n = len(updateList)
    # W type is Variable, which is a inner type defined in cvxpy
    # W.value type is numpy matrix
    W = cv.Variable(n)
    
    cost = sum(cv.entr(W))
    obj = cv.Maximize(cost)
    
    # form the constraints
    constraints = []
    
    # the whole database price 不需要进入whole database price point， 因为有些地方是privacy的，无价的
#     constraints.append(sum([W[i] for i in range(n)]) == databasePoint)
    constraints.append(0 <= W)
    
    constraintList = constructConstraints(updateList, df, queryList, pointList)
    for constraint in constraintList:
        index = constraint[0]
        value = constraint[1]
        
        constraints.append(sum([W[i] for i in constraint[0]]) == constraint[1])
    
    # Form and solve problem.
    prob = cv.Problem(obj, constraints)
    prob.solve() # Returns the optimal value.
    print ("status:", prob.status)
    print ("optimal value", prob.value)
    print ("optimal var", W.value)
    return W.value

In [74]:
display(df)
updateList = updateList2
queryList =[qry1,qry2]
for u in updateList:
    print (str(u))
print (''.join(queryList))
pointList = [10,5]
l = constructConstraints(updateList, df, queryList, pointList)
l
weights = calculateWeight(updateList, df, queryList, pointList)
for index, w in enumerate(weights):
    print (index, ' : ', w)

,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,female,44,fatigue,Advil,cold
1,789,female,68,fatigue,Clofarabine,leukemia
2,739,male,52,fever,Advil,incluenza
3,512,male,73,fatigue,Clofarabine,leukemia


id = 0, row = 0, column = 1, oldValue = female, newValue = female
id = 1, row = 1, column = 2, oldValue = 68, newValue = 44
id = 2, row = 2, column = 5, oldValue = incluenza, newValue = incluenza
id = 3, row = 3, column = 0, oldValue = 512, newValue = 512
id = 4, row = 0, column = 3, oldValue = fatigue, newValue = fatigue
id = 5, row = 1, column = 2, oldValue = 68, newValue = 44
id = 6, row = 2, column = 1, oldValue = male, newValue = female
id = 7, row = 3, column = 0, oldValue = 512, newValue = 512
id = 8, row = 0, column = 1, oldValue = female, newValue = female
id = 9, row = 1, column = 5, oldValue = leukemia, newValue = incluenza
id = 10, row = 2, column = 1, oldValue = male, newValue = female
id = 11, row = 3, column = 3, oldValue = fatigue, newValue = fatigue
id = 12, row = 0, column = 3, oldValue = fatigue, newValue = fatigue
id = 13, row = 1, column = 2, oldValue = 68, newValue = 44
id = 14, row = 2, column = 1, oldValue = male, newValue = female
id = 15, row = 3, column = 2, 

通过 位置 和 price point 来判断， 不通过 query 来判断。

有一种情况，if the query given by the user always gain the equal answer from $D$ and $D'$, 也就是说， 二者总是得到相同的结果， then the unequalList is empty, but one price point attached to it. What should we do for next step? 那就说明，我们的support size 不够大。

计算得到的某个 instance in the support set has a high weight, what does that mean?
就是说， 如果 某个 instance 的weight 算出来比较高， 意味着什么？
考虑一种极端情况， 如果某个 query下， 只过滤出了一个instance， 那么这个query的所有的price point 都被赋给了这个instance， 也就是说这个instance 的weight 会比较高。 也就是，在support set中，这个 query 下只有一个instance 的结果和 original instance结果不一样， 也就是说这个query 问的是一个 common knowledge， 大部分 instance 都知道这个 knowledge， 这个能提供不一样 answer 的 instance 就显得比较重要？ 所以weight 就比较高？

接下来， 通过weight来计算给定 query的 price的时候， 如果要计算的 query 能 filter 上面那个weight 比较高的 instance，那么这个query的price 就会大幅度上升（因为 加上了这个weight）。 


原paper中 optimization 的实现，是：
1. check whether the relation that is modified by the update is involved in the query
2. whether the update tuple contribute to the output. 
3. whether any of the attribute of updated tuple is the projeted attribute in the query. 

In [85]:
## ontology node
import random


class OntologyNode():
    
    """
    the level of root node is 0.
    
    """
    def __init__(self, value):
        self.value = value
        self.children = []
        self.parent = None

    def getValue(self):
        return self.value

    def setValue(self, v):
        self.value = v

    def isLeaf(self):
        tag = False
        if len(self.children) == 0:
            tag = True
        return tag

    def setParent(self, parent):
        self.parent = parent
        self.parent.children.append(self)

    def getParent(self):
        return self.parent

    def addChild(self, child):
        self.children.append(child)

    def getChildren(self):
        """
        get the list of children (node)
        """
        return self.children

    def getLeafNodes(self):
        """
        return a list of leaf values
        """

        result = []

        if self.isLeaf() :
            result.append(self.toString())

        else:
            children = self.getChildren()
            for child in children:
                result.append(child.getLeafNodes())
        return result  ###return values

    def toString(self):
        return self.value

    def getHeight(self):
        """
        get the level of current node. Root node is 0.
        """
        if self.parent is None:
            return 0
        else:
            return self.parent.getHeight()+1

    def getRoot(self):
        if self.parent:
            return self.toString()

        else:
            up = self.getParent()
            return up.getRoot()

def valueConvertToNode(root, value):
    """
    input value, return the corresponding node object, otherwise return none
    
    Args:
        root: the root node of the tree
        value: the value of the node we are looking for
    Return:
        the node object or none.
    """
    if root.value == value:
        return root
    else:
        childrenList = root.getChildren()
        for child in childrenList:
            tmp = valueConvertToNode(child, value)
            if tmp is not None:
                return tmp
            else:
                return None

def getAncestors(root, node):
    """
    given the node, return its all ancestors
    传入node， 得到所有祖先的value list
    
    Args:
        root
        node
    Return:
        a list of ancestor values of this node
    """
    ancestorList = []
    
    def getAncestor(root, node, ancestorList):
        if node.getParent() is None:
            return ancestorList
        else:
            currentParent = node.getParent()
            ancestorList.append(currentParent.getValue())
            getAncestor(root, currentParent, ancestorList)
            return ancestorList
    return getAncestor(root,node,ancestorList)

def getAllLeaf(root, node):
    """
    given the node, return its all leaf node (in the list of value form)
    如果输入的node是 leaf, 那么就直接将该node 输出
    """
    childrenList = []
    
    # help function to maintain the list 
    def getAllLeafHelp(root, node, childrenList):
        if node.isLeaf():
            childrenList.append(node.getValue())
            return childrenList
        else:
            for child in node.getChildren():
#                 childrenList.append(child.getValue())
                getAllLeafHelp(root, child, childrenList)
            return childrenList
    return getAllLeafHelp(root, node, childrenList)

def getAllDescendant(root, node):
    """
    given the node, return its all children (not only leaf node, but all children till leaf)
    如果输入的 node 是leaf, 那么输出就是 none
    """
    childrenList = []
    
    def getAllChildrenHelp(root, node, childrenList):
        """
        如果没有孩子, 就不做操作;
        如果有孩子, 就把孩子加进去, 再递归
        """
        if node.getChildren() is None:
            return childrenList
        else:
            for child in node.getChildren():
                childrenList.append(child.getValue())
                getAllChildrenHelp(root, child, childrenList)
            return childrenList
    return getAllChildrenHelp(root, node, childrenList)

def getNode(root, node, level):
    """
    given the root, node, and level, return the node of the specified level
    if the level is out of the ground node level, then it returns leaf node
    """
    currentLevel = node.getHeight()
    # root is level 0
    levelDifference = currentLevel - level
    for i in range(levelDifference):
        node = node.getParent()
    return node

a = OntologyNode('North America')
b = OntologyNode('Canada')
c = OntologyNode('America')
d = OntologyNode('ON')
e = OntologyNode('BC')
f = OntologyNode('Hamilton')
g = OntologyNode('Toronto')
h = OntologyNode('Vancouver')
i = OntologyNode('Kamloops')
j = OntologyNode('Penticton')
k = OntologyNode('NB')
l = OntologyNode('Baker Brook')
m = OntologyNode('Bathurst')
n = OntologyNode('Dieppe')


b.setParent(a)
c.setParent(a)
d.setParent(b)
e.setParent(b)
k.setParent(b)
f.setParent(d)
g.setParent(d)
h.setParent(e)
i.setParent(e)
j.setParent(e)
l.setParent(k)
m.setParent(k)
n.setParent(k)


root=a

print (' '.join(getAncestors(root, f)))
print (' '.join(getAllLeaf(root, f)))
print (' '.join(getAllChildren(root, d)))
# print(getUpper(root,k,list))
l= getAllChildren(root, d)
print ('l is :', ' '.join([e for e in l]))
assert b.getHeight() == 1 # b is Canada
assert valueConvertToNode(a, "ON") == d # d is ON 

assert getNode(root, f, 2).getValue() == "ON" # f is Hamilton

ON Canada North America
Hamilton
Hamilton Toronto
l is : Hamilton Toronto


                North America
                   /\
                  /  \
             Canada  America
              / | \
             /  |  \
            ON  BC  NB
            /\
           /  \
    Hamilton   Toronto

In [118]:
def getGequeryResult(query, df_local, root, level):
    """
    get the query result at the given level
    
    """
    
    # 首先得到ground level的 result， 这是一个 df 对象 （one column or one cell）
    groundResult = getQueryResult(query, df_local)
    display(groundResult)
    
    # 把该result 先上 uplevel
    (row,column) = groundResult.shape
    print (row,column)
    for i in range(row):
        originalGroundValue = groundResult.iat[i, column-1]
        print(originalGroundValue)
        groundNode = valueConvertToNode(root, originalGroundValue)
        print (groundNode.getValue())
        geValue = getNode(root, groundNode, level).getValue()
        # replace ground value with general value
        groundResult.iat[i,column] = geValue
    return groundResult
result1 = getQueryResult(qry1, df_old)
result1
from utility.ReadJson import reJson
symp=reJson('./data/test1/symp.json')
# n = valueConvertToNode(symp,"fatigue")
symp.getChild

# n.getValue()
# geResult = getGequeryResult(qry1, df_old, symp, 1)


'root'

In [83]:
display(df)
(row, column) = df.shape
row
column
for i in range(row):
    df.iat[i,1] = 'general'
display(df)

for i in range(row):
    df.ix[[i],[1]] = 'change'
display(df)

,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,general,44,fatigue,Advil,cold
1,789,general,68,fatigue,Clofarabine,leukemia
2,739,general,52,fever,Advil,incluenza
3,512,general,73,fatigue,Clofarabine,leukemia


,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,general,44,fatigue,Advil,cold
1,789,general,68,fatigue,Clofarabine,leukemia
2,739,general,52,fever,Advil,incluenza
3,512,general,73,fatigue,Clofarabine,leukemia


,PID,GEN,AGE,SYMP,DRUG,ILLNESS
0,928,change,44,fatigue,Advil,cold
1,789,change,68,fatigue,Clofarabine,leukemia
2,739,change,52,fever,Advil,incluenza
3,512,change,73,fatigue,Clofarabine,leukemia
